In [12]:
from pycalphad import Database, variables as v
from pycalphad.variables import Species

dbf = Database('Al-Cu-Zr_Zhou.tdb')

In [22]:
import itertools
import string
import sympy
import numpy as np

def genericize(mapping, const_array):
    new_array = []
    for subl in const_array:
        new_array.append(tuple(mapping[x] for x in subl))
    return tuple(new_array)
def sigfigs(x, n):
    """Round x to n significant digits"""
    if x != 0:
        return np.around(x, -(np.floor(np.log10(np.abs(x)))).astype(np.int) + (n - 1))
    else:
        return x
def average_piecewise(pw):
    return sum(x.expr for x in pw.args)/len(pw.args)

def categorize_phase(phase_name, constituents, site_ratios):
    if phase_name == 'LIQUID' and len(constituents) == 1:
        return 'liquid'
    elif phase_name == 'GAS' and len(constituents) == 1:
        return 'gas'
    elif len(constituents) == 1 or \
        (len(constituents) == 2 and len(constituents[1]) == 1 and list(constituents[1])[0] == Species('VA')):
        return 'solid'
    elif all(len(x) == 1 for x in constituents):
        return 'intermetallic'
    else:
        return 'complex'

for combo in itertools.combinations(dbf.elements - {'VA', '/-'}, 2):
    combo_str = sorted(set(combo) | {'VA'})
    generic_combo = list(string.ascii_uppercase[:len(combo)])
    combo = [Species(x) for x in combo_str]
    combo_dict = dict(zip([c for c in combo if c.number_of_atoms > 0], string.ascii_uppercase))
    combo_dict[Species('VA')] = 'VA'
    allowed_phases = set()
    # TODO: Categorize each phase? solid solution, intermetallic, complex intermetallic, liquid, etc.
    for phase_name, phase_obj in dbf.phases.items():
        include_phase = True
        for sublattice in phase_obj.constituents:
            if len(set(sublattice).intersection(combo)) == 0:
                # None of the components in a sublattice are active
                # We cannot build a model of this phase
                include_phase = False
                break
        if include_phase:
            allowed_phases |= {phase_name}
    for param in dbf._parameters.all():
        if param['phase_name'] in allowed_phases and param['parameter_type'] in ['G', 'L']:
            active_pure_elements = [list(x.constituents.keys()) for subl in param['constituent_array'] for x in subl]
            active_pure_elements = [el.upper() for constituents in active_pure_elements for el in constituents]
            active_pure_elements = set(active_pure_elements)
            if not active_pure_elements.issubset(combo_str):
                continue
            ratio_sum = sum(dbf.phases[param['phase_name']].sublattices)
            reduced_ratios = tuple(sigfigs(c, 3) for c in dbf.phases[param['phase_name']].sublattices)
            param_val = param['parameter'].subs(dbf.symbols).subs(dbf.symbols)
            # If parameter is Piecewise, average it
            pw_atoms = param_val.atoms(sympy.Piecewise)
            pw_dict = {key: average_piecewise(key) for key in pw_atoms}
            param_val = param_val.subs(pw_dict)
            id_tuple = ([c for c in combo_str if c != 'VA'], param['phase_name'],
                        categorize_phase(param['phase_name'], dbf.phases[param['phase_name']].constituents, reduced_ratios),
                        reduced_ratios, param['parameter_order'],
                        genericize(combo_dict, param['constituent_array']),
                        param_val,
                        param_val.coeff(v.T*sympy.log(v.T))
                       )
            print(id_tuple)

(['CU', 'ZR'], 'LIQUID', 'liquid', (1.0,), 0, (('A',),), -1.94963333333333e-21*T**7 + 1.43581111111111e-8*T**3 - 0.000295204444444444*T**2 - 16.6258213333333*T*log(T) + 89.710867*T + 1938.00955555556 + 52478/(9*T) + 4.0463e+28/T**9, -16.6258213333333)
(['CU', 'ZR'], 'LIQUID', 'liquid', (1.0,), 0, (('B',),), 5.425e-23*T**7 - 0.000486434444444444*T**2 - 21.4153111111111*T*log(T) + 124.729847111111*T - 479.358444444444 + 11657/(3*T) - 1.49210666666667e+30/T**9, -21.4153111111111)
(['CU', 'ZR'], 'LIQUID', 'liquid', (1.0,), 0, (('A', 'B'),), -25.65605*T*log(T) + 222.0803*T - 70319.316, -25.6560500000000)
(['CU', 'ZR'], 'LIQUID', 'liquid', (1.0,), 1, (('A', 'B'),), -4.80625*T*log(T) + 42.49615*T - 11033.106, -4.80625000000000)
(['CU', 'ZR'], 'LIQUID', 'liquid', (1.0,), 2, (('A', 'B'),), 18.4256*T*log(T) - 153.69085*T + 28316.577, 18.4256000000000)
(['CU', 'ZR'], 'LIQUID', 'liquid', (1.0,), 3, (('A', 'B'),), -6.8244*T*log(T) + 59.7719*T - 11184.3605, -6.82440000000000)
(['CU', 'ZR'], 'AL2ZR',